In [18]:
import gensim
from gensim import corpora
from gensim.models import TfidfModel, Phrases
import nltk
from nltk.corpus import reuters
import spacy

nltk.download('reuters')

[nltk_data] Downloading package reuters to
[nltk_data]     C:\Users\rachi\AppData\Roaming\nltk_data...
[nltk_data]   Package reuters is already up-to-date!


True

In [37]:
nlp = spacy.load("en_core_web_sm")
corpus = reuters.sents(categories=["cpu"])

# Convert the corpus to a list of sentences
example_text = [" ".join(sent) for sent in corpus]

texts = []
for document in example_text:
    doc = nlp(document)
    text = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
    texts.append(text)

dictionary = corpora.Dictionary(texts)
corpus_bow = [dictionary.doc2bow(text) for text in texts]

# Create a TF-IDF model
tfidf_model = TfidfModel(corpus_bow)
corpus_tfidf = tfidf_model[corpus_bow]

bigram = Phrases(texts, min_count=1, threshold=1)
texts_with_bigrams = [bigram[line] for line in texts]

trigram = Phrases(texts_with_bigrams, min_count=1, threshold=1)
texts_with_trigrams = [trigram[line] for line in texts_with_bigrams]

print("Bigrams:")
for text in texts_with_bigrams[:7]:
    print(text)

print("\nTrigrams:")
for text in texts_with_trigrams[:7]:
    print(text)

Bigrams:
['u_s', 'INDUSTRIAL_CAPACITY', 'USE_RATE', '81_2', 'PCT_SEPTEMBER', 'UNCHANGED_AUGUST']
['u_s', 'INDUSTRIAL_CAPACITY', 'USE_RATE', '81_2', 'PCT_SEPTEMBER', 'UNCHANGED_AUGUST']
['CANADA', 'MANUFACTURING', 'UTILIZATION', 'rate', 'rise', 'utilization', 'canadian', 'manufacturing', 'capacity', 'rise', '77', '2_pct', 'fourth', 'quarter', '1986', '77', 'pct', 'quarter', 'Statistics', 'Canada', 'say']
['change', 'small', 'mark', 'quarter', 'quarter', '1985', 'utilization', 'rate', 'manufacturing', 'rise', 'federal', 'agency', 'say']
['increase', 'residential', 'construction', 'lead', 'strong', 'increase', 'building', 'material', 'sector', 'lead', '3', '3_pct', 'increase', 'non', 'metallic', 'mineral', 'industry']
['u_s', 'INDUSTRIAL_CAPACITY', 'USE_RATE', 'ROSE_79', '8_PCT', 'FEB_79', '6_PCT', 'JAN']
['u_s', 'INDUSTRIAL_CAPACITY', 'USE_RATE', 'ROSE_79', '8_PCT', 'FEB_79', '6_PCT', 'JAN']

Trigrams:
['u_s_INDUSTRIAL_CAPACITY', 'USE_RATE_81_2', 'PCT_SEPTEMBER_UNCHANGED_AUGUST']
['u_s_I

In [20]:
categories = reuters.categories()
print(categories)

['acq', 'alum', 'barley', 'bop', 'carcass', 'castor-oil', 'cocoa', 'coconut', 'coconut-oil', 'coffee', 'copper', 'copra-cake', 'corn', 'cotton', 'cotton-oil', 'cpi', 'cpu', 'crude', 'dfl', 'dlr', 'dmk', 'earn', 'foorball', 'football', 'fuel', 'gas', 'gnp', 'gold', 'grain', 'groundnut', 'groundnut-oil', 'heat', 'hog', 'housing', 'income', 'instal-debt', 'interest', 'ipi', 'iron-steel', 'jet', 'jobs', 'l-cattle', 'lead', 'lei', 'lin-oil', 'livestock', 'lumber', 'meal-feed', 'money-fx', 'money-supply', 'naphtha', 'nat-gas', 'nickel', 'nkr', 'nzdlr', 'oat', 'oilseed', 'orange', 'palladium', 'palm-oil', 'palmkernel', 'pet-chem', 'platinum', 'potato', 'propane', 'rand', 'rape-oil', 'rapeseed', 'reserves', 'retail', 'rice', 'rubber', 'rye', 'ship', 'silver', 'sorghum', 'soy-meal', 'soy-oil', 'soybean', 'sport', 'strategic-metal', 'sugar', 'sun-meal', 'sun-oil', 'sunseed', 'tea', 'tin', 'trade', 'veg-oil', 'wheat', 'wpi', 'yen', 'zinc']
